# 回归Regression

# 1. Udacity内容
## Regression:   
- Supervised Learning: Take examples of inputs and outputs. Now, given a new input, predict its output. 
- Mapping continous inputs to outputs. (discrete, continous)

## Regression & Function Approximation:
例子child Height和函数逼近或回归有什么关系呢？
向平均值回落的回归概念与逼近函数、输入到输出的映射有什么关系？

例子Child Height是19世纪后期的研究成果，它将所有这些不同的量（父母的身高，人群的平均身高，目标子女的身高）巧妙地联系了起来。   
即用这种函数的方式在它们之间创建了关系。  
于是人们说“这真不错，我们要使用回归这个概念”（Regression to Mean）。    
但他们开始使用后，真正的用意并非向均数回归，而是基于一堆测量点寻找一种数学关系，所以这个词在后来被误用了。  
而我们现在依然在使用这个词。   
所以，现在**回归并不是指向平均值回落，而是使用函数形式来逼近一堆数据点。**


## Regression in Machine Learning:

## Finding the best constant function:
找到可以逼近所有点的最佳的常数。   
**常数函数**：$f(x) = c$    
**误差函数**：$E(c) = \sum_{i=1}^{n}(y_{i} - c)^{2}$ ，LOSS ERROR.    
想要求使得LOSS ERROR最小的常数项，对误差函数求导即可。   
**求导计算过程**：
    $\frac{dE(c)}{dc} = \sum_{i=1}^{n} 2*(y_{i} -c)*(-1)$     
想要找到最小值，使得上式等于0即可。   
    即$\sum_{i=1}^{n} 2*(y_{i} -c)*(-1) = 0$    
    $-\sum_{i=1}^{n}2*(y_{i}-c) = 0$    
    $\sum_{i=1}^{n}y_{i} - \sum_{i=1}^{n}c=0$   
    $n*c = \sum_{i=1}^{n}y_{i}$    
    $c = \frac{\sum_{i=1}^{n}y_{i}}{c}$
    
 根据计算结果，最佳的常数是所有y的平均值。
 
 ## Order of Polynomial多项式的阶：
 k=0: constant    
 k=1: line     
 k=2: parabola    
     $f(x) = c_{0} + c_{1}*x + c_{2}*x^{2} + ... + c_{k}*x^{k}$  
 
 ![title](../images/regression_1.png)